In [5]:
import polars as pl
from op_datasets.etl.ingestion import construct_tasks, reader
from op_datasets.logic.transforms.daily_address_summary import (
    daily_address_summary,
    CONDITIONS,
)
from op_coreutils.logger import structlog

from datetime import datetime
import os

os.environ["OPLABS_ENV"] = "PROD"


log = structlog.get_logger()

In [ ]:
tasks = construct_tasks(chains=["op"], block_spec="126071012:+500")
task = tasks[0]

reader(task, source_spec="goldsky", dataset_names=["blocks", "transactions"])

In [ ]:
dataframes = task.input_dataframes

dataframes

In [ ]:
# check if our blocks are in the range we want

log.info(f"Min block number: {dataframes['blocks']['number'].min()}")
log.info(f"Max block number: {dataframes['blocks']['number'].max()}")
log.info(
    f"Min block number: {datetime.fromtimestamp(dataframes['blocks']['timestamp'].min())}"
)
log.info(
    f"Max block number: {datetime.fromtimestamp(dataframes['blocks']['timestamp'].max())}"
)

In [11]:
result = daily_address_summary(
    dataframes["transactions"],
    conditions=CONDITIONS,
    address="from_address",
    chain_id="chain_id",
    chain="chain",
    dt="dt",
)

In [ ]:
result.head()

In [14]:
# overwrite_partitions(result, "temp", "daily_address_summary", expiration_days=999)

In [15]:
# create test dfs
test_single_tx_df = dataframes["transactions"].filter(
    dataframes["transactions"]["hash"]
    == "0xc64c1471f38b0444d143011c0261a6c14d76a8b32115d39a3f3bf11af337eb55"
)
test_single_tx_df.write_parquet(
    "../../tests/op_datasets/test_daily_address_summary/test_daily_address_summary_single_tx.parquet"
)

test_multiple_txs_df = dataframes["transactions"].filter(
    dataframes["transactions"]["from_address"]
    == "0xd18a8be700b1fc5f7df8a3dbd772fbdecb16ee3a"
)

test_multiple_txs_df.write_parquet(
    "../../tests/op_datasets/test_daily_address_summary/test_daily_address_summary_multiple_txs.parquet"
)

In [16]:
test_result = daily_address_summary(
    test_single_tx_df,
    conditions=CONDITIONS,
    address="from_address",
    chain_id="chain_id",
    chain="chain",
    dt="dt",
)

In [ ]:
test_result

In [ ]:
test_df = dataframes["transactions"].filter(
    dataframes["transactions"]["hash"]
    == "0xc64c1471f38b0444d143011c0261a6c14d76a8b32115d39a3f3bf11af337eb55"
)
test_df

In [19]:
receipt_l1_gas_used = pl.select(test_df["receipt_l1_gas_used"]).item()
receipt_l1_base_fee_scalar = pl.select(test_df["receipt_l1_base_fee_scalar"]).item()
receipt_l1_gas_price = pl.select(test_df["receipt_l1_gas_price"]).item()

In [ ]:
(receipt_l1_gas_used * receipt_l1_base_fee_scalar * receipt_l1_gas_price) / 1e18

In [ ]:
for i in list(test_multiple_txs_df["hash"]):
    print(i)